In [ ]:
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd

In [ ]:
URL = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged=1'

response = requests.get(URL)

In [ ]:
response.status_code

In [ ]:
soup = BS(response.text)

In [ ]:
print(soup.prettify())

## Q1. List of headliners at upcoming events.

#### Grabs lines with that have both the 'a' tag and have the 'class' attribute equaling 'tribe-event-url'.

In [ ]:
events = soup.findAll('a', attrs={'class' : 'tribe-event-url'})

#### Goes through each pulled tag and grabs the title attribute, which happens to be the headliner for the event.

In [ ]:
headliners = [x.get('title') for x in events]
headliners

## Q2. Two lists, one for the date and one for the time of each event.

In [ ]:
time = soup.findAll('time')
time

#### gets all the datetime data for each show

In [ ]:
datetime = [x.get('datetime') for x in time]
datetime

#### splits date and time into seperate values

In [ ]:
date_and_time = [x.split(' ', 1) for x in datetime]
date_and_time

## Q3. Making everything into a Dataframe.

#### date and time data frame

In [ ]:
dat_df = pd.DataFrame(date_and_time, columns=('date', 'time'))
dat_df

#### Headliners Dataframe

In [ ]:
hl_df = pd.DataFrame(headliners, columns=['Artists'])
hl_df

## Q4. Scraping the first 5 pages.

#### URL with the number cut so that in can be changed in loop

In [ ]:
URL_mp = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='

#### Code that scrapes the first 5, makes 2 df of shows and datetime, then merges into one df.

In [ ]:
for pages in range(1, 6):
    req = requests.get(URL_mp + str(pages))
    page_soup = BS(req.text, "html.parser")
    container_headliner = page_soup.findAll('a', attrs={'class' : 'tribe-event-url'})
    container_date = page_soup.findAll('time')
    shows_df = pd.DataFrame([x.get('title') for x in container_headliner], columns=['Artists'])
    dates_df = pd.DataFrame([x.get('datetime') for x in container_date], columns=['Date_and_Time'])
    print(shows_df.merge(dates_df, left_index=True, right_index=True))